<a href="https://colab.research.google.com/github/AUT-Student/NN-HW5/blob/main/NN_HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix

import tensorflow as tf
import tensorflow.keras as keras
from keras.layers import Dense, Input, RNN
import keras.backend as K
from keras.models import Sequential
from keras.callbacks import EarlyStopping

# Dataset

In [2]:
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp1.data
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp2.data
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp3.data
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp4.data
!gdown https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp5.data

Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp1.data
To: /content/lp1.data
100% 27.3k/27.3k [00:00<00:00, 488kB/s]
Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp2.data
To: /content/lp2.data
100% 14.6k/14.6k [00:00<00:00, 257kB/s]
Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp3.data
To: /content/lp3.data
100% 14.5k/14.5k [00:00<00:00, 267kB/s]
Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp4.data
To: /content/lp4.data
100% 33.6k/33.6k [00:00<00:00, 570kB/s]
Downloading...
From: https://archive.ics.uci.edu/ml/machine-learning-databases/robotfailure-mld/lp5.data
To: /content/lp5.data
100% 49.2k/49.2k [00:00<00:00, 394kB/s]


In [3]:
normal_label_datasets = ["normal","normal","ok","normal","normal"]

dataset_x = []
dataset_y = []
for i in range(5):
  normal_label = normal_label_datasets[i]
  dataset_i = pd.read_csv(f"/content/lp{i+1}.data", header=None)
  
  new_label = None
  new_data = None

  for j, row in dataset_i.iterrows():
    if j%16==0:
      new_data = []
      
      if row[0] == normal_label:
         new_label = 0
      else:
         new_label = 1

    else:
      new_data.append([float(item) for item in row[0].split()])

      if j%16==15:
        dataset_x.append(new_data)
        dataset_y.append(new_label)

In [4]:
dataset_x, dataset_y = shuffle(dataset_x, dataset_y, random_state=0)

In [5]:
dataset_len = len(dataset_x)

In [6]:
train_x = dataset_x[:int(0.7*dataset_len)]
train_y = dataset_y[:int(0.7*dataset_len)]

valid_x = dataset_x[int(0.7*dataset_len):int(0.8*dataset_len)]
valid_y = dataset_y[int(0.7*dataset_len):int(0.8*dataset_len)]

test_x = dataset_x[int(0.8*dataset_len):]
test_y = dataset_y[int(0.8*dataset_len):]

In [7]:
sum(train_y) * 100 / len(train_y)

71.60493827160494

In [8]:
sum(test_y) * 100 / len(test_y)

77.41935483870968

# Elman

In [9]:
class ElmanCell(keras.layers.Layer):
  # Refrences: https://github.com/keras-team/keras/blob/v2.9.0/keras/layers/rnn/simple_rnn.py#L242-L492

  def __init__(self, units):
      super().__init__()
      self.units = units
      self.state_size = units

  def build(self, input_shape):
      self.kernel = self.add_weight(shape=(input_shape[-1], self.units), initializer="glorot_uniform", name="kernel")
      self.recurrent_kernel = self.add_weight(shape=(self.units, self.units), initializer="orthogonal", name="recurrent_kernel")
      self.bias = self.add_weight(shape=(self.units), initializer="zeros", name="bias")
      self.built = True

  def call(self, inputs, states):
      previous_hidden = states[0]
      current_hidden = K.dot(inputs, self.kernel) + K.dot(previous_hidden, self.recurrent_kernel)
      current_hidden = K.bias_add(current_hidden, self.bias)
      current_hidden = K.sigmoid(current_hidden)
      return current_hidden, [current_hidden]

In [10]:
class ElmanNetwork(keras.Model):
  def __init__(self, cell_number):
    super().__init__()
    self.model = Sequential([
                             Input((15,6,)),
                             RNN(ElmanCell(cell_number)),
                             Dense(2, activation="softmax")
                            ])
  def call(self, inputs):
    return self.model.call(inputs)

In [27]:
elman = ElmanNetwork(64)

In [ ]:
es_callback = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

In [28]:
elman.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [29]:
elman.fit(x=train_x, y=train_y, epochs=100, batch_size=16, validation_data=(valid_x, valid_y))

Epoch 1/100
21/21 [==============================] - 3s 24ms/step - loss: 0.5801 - accuracy: 0.6574 - val_loss: 0.5586 - val_accuracy: 0.6304
Epoch 2/100
21/21 [==============================] - 0s 9ms/step - loss: 0.4517 - accuracy: 0.7346 - val_loss: 0.5308 - val_accuracy: 0.5435
Epoch 3/100
21/21 [==============================] - 0s 9ms/step - loss: 0.4174 - accuracy: 0.7778 - val_loss: 0.5127 - val_accuracy: 0.5652
Epoch 4/100
21/21 [==============================] - 0s 9ms/step - loss: 0.3940 - accuracy: 0.8086 - val_loss: 0.4926 - val_accuracy: 0.6522
Epoch 5/100
21/21 [==============================] - 0s 7ms/step - loss: 0.3751 - accuracy: 0.8117 - val_loss: 0.4884 - val_accuracy: 0.6957
Epoch 6/100
21/21 [==============================] - 0s 8ms/step - loss: 0.3576 - accuracy: 0.8272 - val_loss: 0.4740 - val_accuracy: 0.6957
Epoch 7/100
21/21 [==============================] - 0s 8ms/step - loss: 0.3467 - accuracy: 0.8364 - val_loss: 0.4655 - val_accuracy: 0.7174
Epoch 8/100


In [30]:
elman.evaluate(x=train_x, y=train_y)
elman.evaluate(x=valid_x, y=valid_y)
elman.evaluate(x=test_x, y=test_y)

3/3 [==============================] - 0s 6ms/step - loss: 0.3403 - accuracy: 0.9355


[0.34029626846313477, 0.9354838728904724]

# Jordan

In [34]:
class JordanCell(keras.layers.Layer):
  # Refrences: https://github.com/keras-team/keras/blob/v2.9.0/keras/layers/rnn/simple_rnn.py#L242-L492

  def __init__(self, units):
      super().__init__()
      self.units = units
      self.state_size = 2

  def build(self, input_shape):
      self.kernel = self.add_weight(shape=(input_shape[-1], self.units), initializer="glorot_uniform", name="kernel")
      self.recurrent_kernel = self.add_weight(shape=(2, self.units), initializer="orthogonal", name="recurrent_kernel")
      self.bias = self.add_weight(shape=(self.units), initializer="zeros", name="bias")
      
      self.output_kernel = self.add_weight(shape=(self.units, 2), name="output_kernel")
      self.output_bias = self.add_weight(shape=(2,), initializer="zeros", name="output_bias")

      self.built = True

  def call(self, inputs, states):
      previous_output = states[0]
      current_hidden = K.dot(inputs, self.kernel) + K.dot(previous_output, self.recurrent_kernel)
      current_hidden = K.bias_add(current_hidden, self.bias)
      current_hidden = K.sigmoid(current_hidden)

      current_output = K.dot(current_hidden, self.output_kernel)
      current_output = K.bias_add(current_output, self.output_bias)
      current_output = K.softmax(current_output)

      return current_output, [current_output]

In [35]:
class JordanNetwork(keras.Model):
  def __init__(self, cell_number):
    super().__init__()
    self.model = Sequential([
                             Input((15,6,)),
                             RNN(JordanCell(cell_number)),
                             Dense(2, activation="softmax")
                            ])
  def call(self, inputs):
    return self.model.call(inputs)

In [36]:
jordan = JordanNetwork(64)

In [38]:
jordan.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [39]:
jordan.fit(x=train_x, y=train_y, epochs=100, batch_size=16, validation_data=(valid_x, valid_y))

Epoch 1/100
21/21 [==============================] - 3s 34ms/step - loss: 0.8110 - accuracy: 0.3364 - val_loss: 0.7384 - val_accuracy: 0.4130
Epoch 2/100
21/21 [==============================] - 0s 9ms/step - loss: 0.6933 - accuracy: 0.5957 - val_loss: 0.6851 - val_accuracy: 0.6522
Epoch 3/100
21/21 [==============================] - 0s 12ms/step - loss: 0.6569 - accuracy: 0.7315 - val_loss: 0.6635 - val_accuracy: 0.7174
Epoch 4/100
21/21 [==============================] - 0s 9ms/step - loss: 0.6362 - accuracy: 0.7191 - val_loss: 0.6525 - val_accuracy: 0.5870
Epoch 5/100
21/21 [==============================] - 0s 10ms/step - loss: 0.6232 - accuracy: 0.7315 - val_loss: 0.6428 - val_accuracy: 0.6304
Epoch 6/100
21/21 [==============================] - 0s 10ms/step - loss: 0.6117 - accuracy: 0.7346 - val_loss: 0.6334 - val_accuracy: 0.7174
Epoch 7/100
21/21 [==============================] - 0s 10ms/step - loss: 0.6001 - accuracy: 0.7685 - val_loss: 0.6239 - val_accuracy: 0.7391
Epoch 8/

In [26]:
test_predict = [np.argmax(item) for item in jordan.predict(test_x, batch_size=1)]

In [27]:
confusion_matrix(test_y, test_predict)

array([[ 4, 17],
       [ 9, 63]])